In [129]:
# To do:
# Fix Dates, must be another published on vs premiered on discrepancy -- OK
# Fix Titles -- OK
# Looks like missing one (1) video for Gibi 344/345 -- OK
# DUPLICATES -- OK
# EDGE CASES ▶ -- OK
# User entered date range -- OK
# Better commenting --
# Duration --
# Flask --
# JS --

# Import Dependencies
from bs4 import BeautifulSoup as bs
import requests
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import time
pd.set_option('display.max_rows', 500)

In [130]:
# Convert Date from Jan 1, 1999 format to datetime object
converted_date = ""
raw_months = {"Jan": 1, "Feb": 2, "Mar" : 3, "Apr" : 4, 
              "May" : 5, "Jun" : 6, "Jul" : 7, "Aug" : 8,
              "Sep" : 9, "Oct" : 10, "Nov" : 11, "Dec" : 12}


def convertDate(raw_date):
    
    try:
        converted_date = ""

        number_month = raw_months.get(raw_date[0])

        date_str = (str(number_month) + "/" + raw_date[1] + "/" + raw_date[2]).replace(",", "")

        converted_date = datetime.strptime(date_str, '%m/%d/%Y')

        return converted_date
    
    except:
        print("Input date is not valid.")

In [131]:
# Get Youtuber's Name
input_name = input("Enter Youtuber's Name: ")
print(input_name)

input_date_range = input("How far back in time do you want to go? (YYYY-MM-DD) or (all-time): ")
print(input_date_range)

if input_date_range == "all-time":
    converted_input_date = datetime.strptime("1950-01-01", '%Y-%m-%d')
    
# elif input_date_range != "all-time" and type(input_date_range) == str:
#     print("This is not a valid input date")
    
else:
    converted_input_date = datetime.strptime(input_date_range, '%Y-%m-%d')

list_name = input_name.split()

converted_name = input_name

if len(list_name) > 1:
    
    converted_name = ""
    
    for i in range(len(list_name)):

        converted_name = converted_name + list_name[i]
        
        if i != len(list_name)-1:
            converted_name = converted_name + "+"

search_name = converted_name

start_url = "https://www.youtube.com/results?search_query=" + search_name

Enter Youtuber's Name: mars phobos remix
mars phobos remix
How far back in time do you want to go? (YYYY-MM-DD) or (all-time): all-time
all-time


In [132]:
print(start_url)
get_youtube_url_response = requests.get(start_url)

https://www.youtube.com/results?search_query=mars+phobos+remix


In [133]:
youtube_name_soup = bs(get_youtube_url_response.text, "lxml")

In [134]:
raw_youtube_name_link = youtube_name_soup.find_all("div", class_="yt-lockup-byline")[0].a.get("href")

videos_link = "https://www.youtube.com" + raw_youtube_name_link + "/videos"

about_link = "https://www.youtube.com" + raw_youtube_name_link + "/about"

print(videos_link)
print(about_link)

https://www.youtube.com/user/TorrentWhore/videos
https://www.youtube.com/user/TorrentWhore/about


In [135]:
# Get About Information
about_html = requests.get(about_link)

# Parse HTML
about_soup = bs(about_html.text, "lxml")

In [136]:
# Artist Information
artist_name = about_soup.find("meta", property="og:title").get("content")

subscribers = about_soup.find_all("span", class_="about-stat")[0].text
subscribers_int = int(subscribers.split(" ")[0].replace(",",""))

total_views = about_soup.find_all("span", class_="about-stat")[1].text
total_views_int = int(total_views[3:len(total_views)].split(" ")[0].replace(",",""))

joined = about_soup.find_all("span", class_="about-stat")[2].text
joined_temp = joined.split(" ")[1:4]
joined_convert = convertDate(joined_temp)

print(f"Artist: {artist_name}")
print(f"Subscribers: {subscribers_int}")
print(f"Views: {total_views_int}")
print(f"Joined: {joined_convert}")

Artist: Mars Phobos
Subscribers: 37
Views: 50956
Joined: 2006-08-14 00:00:00


In [137]:
# Convert User Name to UU Format
youtube_code = raw_youtube_name_link.split("/")[2]

if youtube_code[0:2] == "UC":
    
    youtube_code = raw_youtube_name_link.split("/")[2]

    playlist_link = "https://www.youtube.com" + "/playlist?list=UU" + youtube_code[2:] 

elif youtube_code[0:2] != "UC":
    
    youtube_code_raw = about_soup.find("link", rel="canonical").get("href")
    
    youtube_code = youtube_code_raw.split("/")[4]
    
    playlist_link = "https://www.youtube.com" + "/playlist?list=UU" + youtube_code[2:]  
    

print(playlist_link)

https://www.youtube.com/playlist?list=UUWOKPH3jOLZhZiKQMjl1Fmg


In [138]:
# Get Playlist Response
playlist_response = requests.get(playlist_link)

In [139]:
# Create Playlist Soup Object
playlist_soup = bs(playlist_response.text, 'lxml')

In [140]:
# Get First Video URL as Starting Point
first_video = "https://www.youtube.com" + playlist_soup.find_all("a", class_="pl-video-title-link")[0].get("href").split("&")[0]
first_video_within_playlist = first_video + "&" + playlist_link.split("?")[1]

print(first_video_within_playlist)

https://www.youtube.com/watch?v=Qu0zMoJt8Q4&list=UUWOKPH3jOLZhZiKQMjl1Fmg


In [141]:
# Create Soup Object for First Video Inside Playlist
playlist_inside_request = requests.get(first_video_within_playlist) 

playlist_inside_soup = bs(playlist_inside_request.text, "lxml")

In [142]:
urls_all = []
total_videos_in_playlist = int(playlist_inside_soup.find("span", id="playlist-length").text.replace(" videos","").replace(",",""))
number_of_videos_in_page = len(playlist_inside_soup.find_all("span", class_="index")) 
last_video_index = int(playlist_inside_soup.find_all("span", class_="index")[-1].text.replace("\n        ","").replace("\n    ",""))
last_shown_link = playlist_inside_soup.find_all("span", class_="index")[-1].find_next("a").get("href")
link_fix = "https://www.youtube.com" + last_shown_link
    
for i in range(total_videos_in_playlist):   

    if i == 0:
        
#        print(f"loop1, i={i}")
        
        first_link = playlist_inside_soup.find("span", class_="index", text=f"\n        ▶\n    ")
        
        url = "https://www.youtube.com" + first_link.find_next("a").get("href")
        
        original_url = url.split("&")[0]

        urls_all.append(original_url)
        
        next_link = first_link
        
    elif i == last_video_index:

#        print(f"loop2, i={i}------------------------------------------------")
        
        playlist_inside_request = requests.get(link_fix)
        
        playlist_inside_soup = bs(playlist_inside_request.text, "lxml")
        
        last_shown_link = playlist_inside_soup.find_all("span", class_="index")[-1].find_next("a").get("href")
        
        link_fix = "https://www.youtube.com" + last_shown_link
        
        last_video_index = int(playlist_inside_soup.find_all("span", class_="index")[-1].text.replace("\n        ","").replace("\n    ",""))
        
        first_link = playlist_inside_soup.find("span", class_="index", text=f"\n        {i+1}\n    ")

        if first_link is None:
            
            next_link = playlist_inside_soup.find("span", class_="index", text=f"\n        ▶\n    ")
            
        else:
            
            next_link = first_link
        
        next_url = "https://www.youtube.com" + next_link.find_next("a").get("href")
        
        original_url = next_url.split("&")[0]
        
        urls_all.append(original_url)
        
        number_of_videos_in_page = len(playlist_inside_soup.find_all("span", class_="index")) - 1
              
    else:
        
#        print(f"loop3, i={i}")
 
        if i == 1:

            first_link = playlist_inside_soup.find("span", class_="index", text=f"\n        ▶\n    ")
            
        elif playlist_inside_soup.find("span", class_="index", text=f"\n        {i}\n    ") is None:
            
            first_link = playlist_inside_soup.find("span", class_="index", text=f"\n        ▶\n    ")
        
        else:
        
            first_link = playlist_inside_soup.find("span", class_="index", text=f"\n        {i}\n    ")
            
#         print(f"{first_link}")

        next_link = first_link
        
#         print(f"{next_link}")

        next_link = next_link.find_next("span", class_="index")

        next_url = "https://www.youtube.com" + next_link.find_next("a").get("href")

        original_url = next_url.split("&")[0]

        urls_all.append(original_url)
        
    print(f"({i+1}/{total_videos_in_playlist})")


(1/6)
(2/6)
(3/6)
(4/6)
(5/6)
(6/6)


In [143]:
# Going to Each Video and Extracting Data
published_on = []
raw_published_on = []
views = []
date = []
duration_videos = []
likes = []
dislikes = []
title_videos = []
categories = []

for i in range(len(urls_all)):
    
    video_url = urls_all[i]
    video_response = requests.get(video_url)
    video_soup = bs(video_response.text, 'lxml')
    
    # Publish Date
    raw_publish_date = video_soup.find("div", id="watch-uploader-info").text
    raw_published_on.append(raw_publish_date)
    
        # Handle All Raw Dates "Premiered", ""Published", "Streamed"
    publish_date_format = raw_publish_date.split(" ")[len(raw_publish_date.split(" "))-3:len(raw_publish_date.split(" "))]
    publish_date_convert = convertDate(publish_date_format)
    
    # Break if Date Less than Input Date Range
    if publish_date_convert < converted_input_date:
        break
    
    else:
        published_on.append(publish_date_convert)
    
    # Title
    title = video_soup.find("title").text.replace(" - YouTube", "")
    title_videos.append(title)
    
    # Views
    string_views = video_soup.find("div", id="watch7-views-info").text.replace(" views", "").replace(",","").replace("\n","")
    int_views = int(string_views)
    views.append(int_views)
    
#     # Duration
#     duration = video_soup.find("span", class_="video-time").text
#     duration_videos.append(duration)
    
    # Likes
    string_likes = video_soup.find("button", title="I like this").text
    if string_likes != "":
        int_likes = int(string_likes.replace(",",""))
        likes.append(int_likes)
    else:
        likes.append(0)
    
    # Dislikes
    string_dislikes = video_soup.find("button", title="I dislike this").text
    if string_dislikes != "":
        int_dislikes = int(string_dislikes.replace(",",""))
        dislikes.append(int_dislikes)
    else:
        dislikes.append(0)
    
    # Category
    category = video_soup.find("h4", class_="title", text="\n      Category\n    ").find_next("a").text
    categories.append(category)
    
    print(f"({i+1}/{len(urls_all)})")
    
urls_to_date = urls_all[0:len(published_on)]

(1/6)
(2/6)
(3/6)
(4/6)
(5/6)
(6/6)


In [144]:
# Create DataFrame
df = pd.DataFrame({"Artist" : artist_name,
                   "Joined" : joined_convert,
                   "Subscribers" : subscribers_int,
                   "Total Views" : total_views_int,
                   "Date": published_on,
                   "Title" : title_videos,
                   "Category" : categories,
                   #"Duration" : duration_videos,
                   "Views" : views,
                   "Likes" : likes,
                   "Dislikes" : dislikes,
                   "URL" : urls_to_date,
                  })

# df = df.sort_values("Date",ascending=False).reindex()
df.to_csv(f"{artist_name}_scrape.csv")
df

,Artist,Joined,Subscribers,Total Views,Date,Title,Category,Views,Likes,Dislikes,URL
0,Mars Phobos,2006-08-14,37,50956,2016-10-23,Everything But The Girl - Missing (Mars Phobos...,Music,174,2,0,https://www.youtube.com/watch?v=Qu0zMoJt8Q4
1,Mars Phobos,2006-08-14,37,50956,2015-09-28,Justin Bieber - What Do You Mean? (Mars Phobos...,Music,159,1,1,https://www.youtube.com/watch?v=rzHSG9cwHjg
2,Mars Phobos,2006-08-14,37,50956,2014-12-26,"Ariana Grande, The Weeknd - Love Me Harder (Ma...",Music,216,3,0,https://www.youtube.com/watch?v=VVwQgfU9XDQ
3,Mars Phobos,2006-08-14,37,50956,2013-09-10,Tiësto ft. Kyler England - Take Me (Charon Noi...,Music,459,10,0,https://www.youtube.com/watch?v=WHUQbPYcB5k
4,Mars Phobos,2006-08-14,37,50956,2013-04-18,Daft Punk - Get Lucky (Mastered) Official Feat...,Music,47747,460,37,https://www.youtube.com/watch?v=HIpfdSrGF5k
5,Mars Phobos,2006-08-14,37,50956,2013-03-17,deadmau5 and Imogen Heap - Telemiscommunicatio...,Music,2194,44,2,https://www.youtube.com/watch?v=TlJbVua3NdY
